In [4]:
import requests
from bs4 import BeautifulSoup
import os
import re
from urllib.parse import urlparse, parse_qs


def sanitize_filename(name):
    """
    OS에서 사용할 수 없는 문자 제거
    """
    return re.sub(r'[\\/:*?"<>|]', '_', name)


def get_webtoon_title_and_episode(webtoon_url):
    """
    웹툰 상세 페이지에서 제목과 회차 번호 추출
    """
    res = requests.get(webtoon_url)
    res.encoding = 'utf-8'
    if not res.ok:
        print(f"[에러] 웹툰 페이지 요청 실패: {res.status_code}")
        return None, None, None

    soup = BeautifulSoup(res.text, 'html.parser')

    # 제목 추출: <div class="title_area"><strong>...</strong>
    strong_tag = soup.select_one('div.title_area strong')
    if strong_tag:
        full_title = strong_tag.text.strip()
        title = full_title.split('-')[0].strip()  # "낢이 사는 이야기 - 부제목" → "낢이 사는 이야기"
        title = sanitize_filename(title)
    else:
        title = "제목없음"

    # 회차 번호 추출: URL의 no 파라미터
    parsed_url = urlparse(webtoon_url)
    no_param = parse_qs(parsed_url.query).get('no', ['0'])[0]
    episode = f"{no_param}화"

    return soup, title, episode


def get_image_urls(soup):
    """
    BeautifulSoup 객체에서 본문 이미지 URL 목록 추출
    """
    img_tags = soup.select("img[src*='IMAG01']")
    img_urls = [img['src'] for img in img_tags]
    return img_urls


def create_directory(path):
    """
    디렉토리 생성 (중첩 포함)
    """
    os.makedirs(path, exist_ok=True)


def download_images(img_urls, save_dir, referer_url):
    """
    이미지 다운로드 및 저장
    """
    headers = {'referer': referer_url}

    for img_url in img_urls:
        res = requests.get(img_url, headers=headers)
        if not res.ok:
            print(f"[에러] 이미지 요청 실패: {res.status_code} | {img_url}")
            continue

        file_name = os.path.basename(img_url)
        file_path = os.path.join(save_dir, file_name)

        with open(file_path, 'wb') as f:
            f.write(res.content)
            print(f"[저장 완료] {file_path} ({len(res.content):,} bytes)")


def main():
    # ▶️ 웹툰 상세 페이지 URL 입력
    webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=48&week=tue'

    # 제목, 회차, soup 추출
    soup, title, episode = get_webtoon_title_and_episode(webtoon_url)
    if not soup or not title or not episode:
        print("[중단] 제목 또는 회차 정보를 가져오지 못했습니다.")
        return

    # 디렉토리 생성: img/제목/회차/
    save_dir = os.path.join("img", title, episode)
    create_directory(save_dir)

    print(f"[시작] {title} {episode} 이미지 다운로드...")

    # 이미지 URL 추출
    img_urls = get_image_urls(soup)
    if not img_urls:
        print("[중단] 이미지가 없습니다.")
        return

    # 이미지 다운로드
    download_images(img_urls, save_dir, referer_url=webtoon_url)

    print(f"[완료] {title} {episode} 이미지 다운로드 완료 ✅")


if __name__ == '__main__':
    main()

[시작] 낢이 사는 이야기 48화 이미지 다운로드...
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_1.jpg (149,542 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_2.jpg (130,246 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_3.jpg (151,735 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_4.jpg (160,334 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_5.jpg (58,195 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_6.jpg (132,660 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_7.jpg (100,942 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_8.jpg (144,242 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9e17c599da2a653193f969945e9315_IMAG01_9.jpg (98,487 bytes)
[저장 완료] img\낢이 사는 이야기\48화\20250224122436_df9